In [0]:
%pip install databricks-vectorsearch flashrank databricks_langchain
dbutils.library.restartPython()

In [0]:
# from databricks.vector_search.client import VectorSearchClient
# from databricks.sdk import WorkspaceClient
# from databricks_langchain import DatabricksVectorSearch, DatabricksEmbeddings
# import databricks.sdk.service.catalog as c
# from mlflow.deployments import get_deploy_client
# from flashrank import Ranker, RerankRequest


# vsc = VectorSearchClient(disable_notice=True)
# ranker = Ranker(
#     model_name="rank-T5-flan", cache_dir="/Volumes/dev_appian_poc/00_bronze/artifacts"
# )


# embedding_model = DatabricksEmbeddings(endpoint="databricks-bge-large-en")

# if not vsc.endpoint_exists("appian_vsc_poc"):
#     vsc.create_endpoint(name="appian_vsc_poc")


# if not vsc.index_exists(
#     endpoint_name="appian_vsc_poc",
#     index_name="dev_appian_poc.02_gold.ingested_text_index",
# ):
#     index = vsc.create_delta_sync_index(
#         endpoint_name="appian_vsc_poc",
#         source_table_name="dev_appian_poc.02_gold.ingestion_text_embeddings",
#         index_name="dev_appian_poc.02_gold.ingested_text_index",
#         pipeline_type="TRIGGERED",
#         primary_key="id",
#         embedding_vector_column="embedding",
#         embedding_dimension=1024,
#         embedding_model_endpoint_name="databricks-bge-large-en",
#     )
# else:
#     index = vsc.get_index(
#         endpoint_name="appian_vsc_poc",
#         index_name="dev_appian_poc.02_gold.ingested_text_index",
#     )


# vector_store = DatabricksVectorSearch(
#     index_name="dev_appian_poc.02_gold.ingested_text_index",
#     text_column="content",
#     embedding=embedding_model,
# )

# sim = vector_store.as_retriever(search_kwargs={"k": 2}).invoke("who is robert peng")

# from mlflow.deployments import get_deploy_client
# deploy_client = get_deploy_client("databricks")

# # List available serving endpoints
# endpoints = deploy_client.list_endpoints()
# for endpoint in endpoints:
#     print(f"Endpoint name: {endpoint['name']}")

In [0]:
from databricks.vector_search.client import VectorSearchClient
from databricks.sdk import WorkspaceClient
from databricks_langchain import DatabricksVectorSearch, DatabricksEmbeddings
import databricks.sdk.service.catalog as c
from mlflow.deployments import get_deploy_client
from flashrank import Ranker, RerankRequest


def get_retriever(persist_dir: str = "/Volumes/dev_appian_poc/00_bronze/artifacts", k: int = 2):
    vsc = VectorSearchClient(disable_notice=True)
    embedding_model = DatabricksEmbeddings(endpoint="databricks-bge-large-en")

    if not vsc.index_exists(
        endpoint_name="appian_vsc_poc",
        index_name="dev_appian_poc.02_gold.ingested_text_index",
    ):
        index = vsc.create_delta_sync_index(
            endpoint_name="appian_vsc_poc",
            source_table_name="dev_appian_poc.02_gold.ingestion_text_embeddings",
            index_name="dev_appian_poc.02_gold.ingested_text_index",
            pipeline_type="TRIGGERED",
            primary_key="id",
            embedding_vector_column="embedding",
            embedding_dimension=1024,
            embedding_model_endpoint_name="databricks-bge-large-en",
        )
    else:
        index = vsc.get_index(
            endpoint_name="appian_vsc_poc",
            index_name="dev_appian_poc.02_gold.ingested_text_index",
        )

    vector_store = DatabricksVectorSearch(
        index_name="dev_appian_poc.02_gold.ingested_text_index",
        text_column="content",
        embedding=embedding_model,
        )
    return vector_store.as_retriever(search_kwargs={"k": k})
    

In [0]:
def get_ranked_query(query: str, k: int = 2):
    vs = get_retriever(k=k)
    similar_documents = vs.invoke(query)
    results = [each.page_content for each in similar_documents]
    passages_dicts = [{"text": passage} for passage in results]

    ranker = Ranker(
        model_name="rank-T5-flan",
        cache_dir="/Volumes/dev_appian_poc/00_bronze/artifacts",
    )

    rerankrequest = RerankRequest(query=query, passages=passages_dicts)

    return ranker.rerank(rerankrequest)

In [0]:
rerankedresponse = get_ranked_query("what school did robert peng go to", k=2)

print(rerankedresponse)

In [ ]:
from databricks_langchain import ChatDatabricks

chat_model = ChatDatabricks(endpoint="databricks-meta-llama-3-3-70b-instruct", max_tokens=300)
print(f"test chat model:{chat_model.invoke("Who is robert peng")}")

In [ ]:
from langchain.agents import create_agent
from langchain.agents.middleware import dynamic_prompt, ModelRequest


@dynamic_prompt
def prompt_with_context(request: ModelRequest):
    rerankedresponse = get_ranked_query(request.messages[-1].content, k=1)
    context = "\n\n".join([doc['text'] for doc in rerankedresponse])
    system_prompt = (
        "Use the given context to answer the question. "
        "If you don't know the answer, say you don't know. "
        "Use three sentence maximum and keep the answer concise. "
        f"Context: {context}"  
    )
    return system_prompt



agent = create_agent(model = chat_model, middleware=[prompt_with_context])
agent.invoke({"messages": [{"role": "user", "content": "who is robert peng"}]})
